<a href="https://colab.research.google.com/github/Tugrulyavuzates/Odev/blob/master/Restaurant_recommender_august_19_v2_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task: Generate Meaningful Restaurant Recommendations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
df = pd.read_csv('/content/TripAdvisor_RestauarantRecommendation.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/TripAdvisor_RestauarantRecommendation.csv'

In [ ]:
df.head(10)
#trim "reviews" from No of Reviews
#get the first number from "Reviews" column
#drop "Menu", "Contact Number", "Trip_advisor Url"

In [ ]:
df.columns

In [ ]:
df_copy = df.copy()
df_copy = df_copy.drop(['Menu', 'Contact Number', 'Trip_advisor Url'], axis=1)

In [ ]:
df_copy.head()

In [ ]:
df_copy.isnull().sum()

In [ ]:
print(df_copy.Reviews.unique())

In [ ]:
print(df_copy['No of Reviews'].unique())

In [ ]:
display(df_copy[df_copy.Reviews == 'No review'])

In [ ]:
display(df_copy[df_copy['No of Reviews'] == 'Undefined Number'])

In [ ]:
df_copy = df_copy.drop([1744,2866])
df_copy = df_copy.reset_index(drop=True)
df_copy

In [ ]:
df_copy.Comments = df_copy.Comments.fillna('')

In [ ]:
df_copy.Type = df_copy.Type.fillna(df_copy.Type.value_counts().index[0])

In [ ]:
df_copy.isnull().sum()

In [ ]:
#********* To better see the types of cuisines and do operations on them more easily we append them to a list and then convert that list to a pandas series ***************

types = []
for i in range(len(df_copy)):
  if type(df_copy.Type[i]) == str:
    types.append(df_copy.Type[i].split(','))
flat_list = list(itertools.chain(*types))
series = pd.Series(flat_list)

In [ ]:
ax = series.value_counts()[:10].plot(kind = "pie", shadow = True, cmap = plt.get_cmap('Spectral'), figsize=(10,10))
ax.set_ylabel('Cuisine Types')
plot = ax.set_title('Top 10 Cuisine Types by Entry')
#

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'series' is already defined and contains the flat list of cuisines

# Step 1: Get the top 10 most frequent types
ax = series.value_counts()[:10].plot(
    kind='pie',           # Plot as a pie chart
    shadow=True,          # Add shadow for visual effect
    cmap=plt.get_cmap('Spectral'),  # Use the 'Spectral' colormap
    figsize=(10, 10)      # Set the figure size
)

# Step 2: Customize the plot
ax.set_ylabel('')  # Optional: Remove y-label for a cleaner look
ax.set_title('Top 10 Cuisine Types by Entry')  # Set the title

# Display the plot
plt.show()

In [ ]:
pd.DataFrame({"No of Restaurant serving":series.value_counts()}).head(10)

# List Comprehension
 important!!!


In [ ]:
df_copy['State'] = [i.split(',')[-1].split(" ")[1] for i in df_copy.Location]
df_copy['Zipcode'] = [i.split(',')[-1].split(" ")[-1] for i in df_copy.Location]
df_copy['City'] = [",".join(i.split(",")[:-1]) for i in df_copy.Location] #********************************* Beautifullll for a newbieee!!!!! **************************************
df_copy = df_copy.drop(['Location'], axis=1)

df_copy

In [ ]:
print(df_copy[['City', 'State', 'Zipcode']].isnull().sum())

In [ ]:
df_copy = df_copy.drop(df_copy[df_copy['State']==''].index[0])
df_copy.State.value_counts()

In [ ]:
ax = df_copy.State.value_counts().plot(kind = "bar", color= 'Purple', figsize=(10,5))
ax.set_ylabel('Number of Restaurants')
ax.set_xlabel('States')
plot = ax.set_title('Number of Restaurants in each State')

In [ ]:
pd.DataFrame({'No of Restaurants per State':df_copy.State.value_counts()}).head(10)

C. Ranking the best (most highly rated) restaurants from each state
Now most highly rated restaurant has to include two factors: -
Number of people who reviewed
Review Ratings on 5


In [ ]:
df_copy['Reviews'] = [float(i.split(" ")[0]) for i in df_copy.Reviews]
df_copy['No of Reviews'] = [int(i.split(" ")[0].replace(",","")) for i in df_copy['No of Reviews']]

df_copy

In [ ]:
df_copy = df_copy.drop([1744,2866])

In [ ]:
df_copy['weighted_ratings'] = df_copy['Reviews'] * df_copy['No of Reviews']

Weighted Ratings Calculation:
The weighted_ratings is calculated as follows:

**kalın metin**
df['weighted_ratings'] = df.Reviews * df['No of Reviews']
Reviews: This might represent the average rating given by reviewers.
No of Reviews: This represents the number of reviews a particular item (e.g., product, movie, etc.) received.
Weighted Ratings Explained:
The weighted_ratings is essentially the total score or total rating points calculated by multiplying the average rating by the number of reviews. This gives more weight to ratings that have a larger number of reviews, which makes sense because a rating with more reviews is generally more reliable than a rating based on just a few reviews.
Why the Maximum Might Be Selected:
Given this calculation, selecting the maximum weighted_ratings could make sense in certain contexts:

Best Overall Rating: The maximum weighted_ratings might represent the item that received the most total positive feedback, considering both the number of reviews and the average rating.

Dominant Opinion: Since weighted_ratings is a product of the average rating and the number of reviews, a high value suggests both a high average rating and a significant number of reviews. Therefore, selecting the maximum could highlight the most positively received item.

Is There a Method Like This?
While this approach is not standard in statistical analysis, it is a practical method in scenarios like product reviews, where:

The highest weighted_ratings can represent the product with the most significant overall positive sentiment.
Summary:
Purpose of Maximum: Selecting the maximum weighted_ratings likely aims to identify the item that not only has a high average rating but also a large number of reviewers backing that rating.
Context-Specific Logic: This method isn't about finding a statistical mean but rather about identifying the most robustly positive response within your dataset.
If the analysis is intended to find the item that stands out most in terms of positive feedback (both in terms of quantity and quality), then using the maximum of weighted_ratings is reasonable.

In [ ]:
df_copy['weighted_ratings']

In [ ]:
labels = df_copy.State.unique().flatten()
labels


In [ ]:
average_vote_share_list = [df_copy[df_copy.State == i].weighted_ratings.max() for i in labels]
avg_wt_ratings = pd.DataFrame({'State':labels, 'Average Weighted Ratings':average_vote_share_list})
plot = sns.catplot(x='State', y="Average Weighted Ratings", kind = 'bar', data= avg_wt_ratings, palette ="rocket")
plt.title('Best Restaurant Among States')
plt.gcf().set_size_inches(9, 6)

In [ ]:
df_copy[df_copy['weighted_ratings']==df_copy['weighted_ratings'].max()]

In [ ]:
df_copy[df_copy['No of Reviews'] == df_copy['No of Reviews'].max()]

In [ ]:

total_vote_share_list = [df_copy[df_copy.State == i]['No of Reviews'].sum() for i in labels]
total_vote_share = pd.DataFrame({'State':labels, 'Total Weighted Ratings':total_vote_share_list})
plot = sns.catplot(x='State', y="Total Weighted Ratings", kind = 'bar', data= total_vote_share, palette ="rocket")
plt.title('Best Restaurant Among States')
plt.gcf().set_size_inches(9, 6)

In [ ]:
labels = df_copy['City'].unique().flatten()
total_vote_share_list = [df_copy[df_copy['City']==i].weighted_ratings.sum() for i in labels]
total_wt_ratings = pd.DataFrame({'City':labels, 'Total Weighted Ratings': total_vote_share_list})
total_wt_ratings = total_wt_ratings.sort_values(by=['Total Weighted Ratings'],ascending=False).head(5)
plot = sns.catplot(x='City', y="Total Weighted Ratings", kind="bar", data=total_wt_ratings, palette="cubehelix")
plt.title("Top 5 Cities For Food")
plt.gcf().set_size_inches(9, 6)

In [ ]:
#### Our Main Feature - Comments
df.Comments

In [ ]:
df = df.dropna(subset=['Comments'])


In [ ]:
### Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

### Construct the required tf-idf matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df.Comments)

### Output Shape of tf-idf matrix
tfidf_matrix.shape

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df_copy.index, index= df_copy.Name).drop_duplicates()
indices

In [ ]:
def get_recommendations(name, cosine_sim=cosine_sim):

    ### Index of the restaurant which matches the name
    idx = indices[name]

    ### Get the pairwise similarity
    sim_scores = list(enumerate(cosine_sim[idx]))

    ### Sort the restaurants based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)

    ### Get the similarity scores of the 10 Most similar restuarants
    sim_scores = sim_scores[1:11]

    ### Get the restauarant inidices
    restaurant_indices = [i[0] for i in sim_scores]

    ### Resturn the Top 10 most similar restaurants
    return df['Name'].iloc[restaurant_indices]

In [ ]:
get_recommendations('Coach House Diner')

In [ ]:
# Example: combine cosine similarity with another feature like cuisine type
def get_hybrid_recommendations(name, cosine_sim=cosine_sim, df=df):
    if name not in indices:
        return "Restaurant not found. Please check the name and try again."

    idx = indices[name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]
    restaurant_indices = [i[0] for i in sim_scores]

    recommended_restaurants = df.iloc[restaurant_indices]

    # Optionally, you can filter or rank these further by another feature
    # e.g., Filter by a specific cuisine type or sort by average rating
    # recommended_restaurants = recommended_restaurants[recommended_restaurants['Cuisine'] == 'Italian']
    # recommended_restaurants = recommended_restaurants.sort_values(by='Rating', ascending=False)

    return recommended_restaurants['Name']


In [ ]:
get_hybrid_recommendations('Coach House Diner')